In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import pandas as pd
import textstat
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
from stop_word_list import *
from stop_word_list_empty import *
from tqdm import tqdm
import sys  
import contractions
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker

In [3]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [4]:
spell_data = pd.read_csv('./processing_datasets/preprocessed_word.csv', error_bad_lines=False);
spell_data

C:\Users\researcher\AppData\Local\Temp\ipykernel_5340\3912034103.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  spell_data = pd.read_csv('./processing_datasets/preprocessed_word.csv', error_bad_lines=False);


,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'..."
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '..."
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ..."
...,...,...,...,...,...,...,...,...,...
163842,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",Only about 3 or 4 hours of it was me actually ...,-1,0,22.3,only about or hour of it be me actually enjoy ...,"['only', 'about', 'or', 'hour', 'of', 'it', 'b..."
163843,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The free roam exploration runs dry very quickl...,-1,0,41.1,the free roam exploration run dry very quickly...,"['the', 'free', 'roam', 'exploration', 'run', ..."
163844,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...","Overall, No Man's Sky is a lazy game not even ...",-1,0,9.7,overall no man be sky be a lazy game not even ...,"['overall', 'no', 'man', 'be', 'sky', 'be', 'l..."
163845,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The game's cycle is essentially go to a planet...,-1,0,23.7,the game be cycle be essentially go to a plane...,"['the', 'game', 'be', 'cycle', 'be', 'essentia..."


In [5]:
def clean_content(words):
    spell = SpellChecker()
    spell.word_frequency.load_dictionary('./webster_dictionary.json')
    
    words = eval(words)
    for i,word in enumerate(words):
        new_word = spell.correction(word)
        if word != new_word:
            print(word,"->",new_word)
            print(spell.candidates(word))
            words[i] = spell.correction(word)
    return str(words)
spell_data['correct'] = clean_content
spell_data['correctmapping'] = spell_data['wordtoken'].map(clean_content)

nothis -> nothin
{'nothir', 'sothis', 'nothus', 'notis', 'nothin', 'nothi'}
revist -> resist
{'revuist', 'revisit', 'revict', 'relist', 'revest', 'reist', 'revise', 'resist', 'rexist'}
additon -> addition
{'addison', 'addition'}
somwhat -> somewhat
{'somewhat'}
repetative -> repetitive
{'repetitive', 'reputative'}
pricepoint -> None
None
samey -> same
{'ramey', 'samer', 'mamey', 'jamey', 'savey', 'samel', 'gamey', 'sammy', 'same', 'samy', 'samek', 'samely', 'samen', 'camey'}
heridium -> heredium
{'peridium', 'heredium'}
asimov -> simon
{'arimo', 'simon', 'aimo', 'asimen', 'admov', 'saimon', 'askov', 'asio', 'animo', 'atimon'}
heinlein -> feinleib
{'heublein', 'feinleib'}
heinlein -> feinleib
{'heublein', 'feinleib'}
asimov -> simon
{'arimo', 'simon', 'aimo', 'asimen', 'admov', 'saimon', 'askov', 'asio', 'animo', 'atimon'}
ugprade -> upgrade
{'upgrade'}
multitool -> multi-tool
{'multi-tool'}
achive -> active
{'achieve', 'active', 'chive', 'archive', 'achime'}
amoungst -> amongst
{'among

fromn -> from
{'froma', 'frorn', 'fron', 'frown', 'from', 'fromm', 'frome'}
mantains -> maintains
{'maintains'}
furture -> future
{'future', 'nurture', 'furrure'}
invetory -> inventory
{'inventory'}
skyrim -> skyrin
{'skyrin'}
frieghters -> freighters
{'freighters'}
immpressive -> impressive
{'impressive'}
acutally -> actually
{'actually'}
colelct -> collect
{'collect'}
acutally -> actually
{'actually'}
survivala -> survival
{'survival', 'survivals'}
siutation -> situation
{'scutation', 'situation', 'siltation', 'sputation'}
frieghter -> freighter
{'frighter', 'freighter'}
extemely -> extremely
{'extremely'}
gamechanger -> None
None
basebuilding -> None
None
homeworlds -> homeworld
{'homeworld'}
homeworlds -> homeworld
{'homeworld'}
friegters -> fritters
{'fritters', 'fretters', 'freighters'}
gamemodes -> galeodes
{'galeodes'}
thamium -> thamnium
{'thamnium'}
koolaid -> koolah
{'koolah', 'onlaid', 'koolau'}
suprise -> surprise
{'uprise', 'surprise', 'sunrise', 'supprise'}
undoubtibly -

{'masterpiece'}
screenshots -> screenshot
{'screenshot'}
unfortunetly -> unfortunately
{'unfortunately'}
repetative -> repetitive
{'repetitive', 'reputative'}
upvoted -> unvoted
{'unvoted'}
acheivement -> achievement
{'achievement'}
privatter -> privateer
{'privater', 'privateer'}
bounus -> bonus
{'bounds', 'bunus', 'bonus'}
alot -> lot
{'blot', 'alost', 'slot', 'clot', 'alit', 'abot', 'alout', 'alo', 'lot', 'aot', 'alon', 'salot', 'alto', 'alt', 'aloe', 'alop', 'alod', 'aloft', 'aloo', 'flot', 'alcot', 'allot', 'alow', 'plot', 'ilot'}
galay -> galaxy
{'gally', 'galany', 'galey', 'galaxy', 'galax', 'gamay', 'salay', 'malay', 'galas', 'galan', 'gala', 'palay', 'galah', 'galway', 'alay'}
discovring -> discovering
{'discovering'}
fis -> is
{'fzs', 'fits', 'vis', 'fss', 'fio', 'xis', 'uis', 'zis', 'fts', 'nis', 'fims', 'fcs', 'lis', 'ais', 'fia', 'fips', 'pis', 'fic', 'yis', 'fms', 'cis', 'is', 'wis', 'ffs', 'kis', 'mis', 'firs', 'fim', 'fix', 'fig', 'fi', 'fids', 'bis', 'fris', 'fgs', 'gi

roleplay -> replay
{'foreplay', 'replay', 'rokelay', 'teleplay', 'roleplayed'}
pefect -> perfect
{'refect', 'prefect', 'defect', 'perfect'}
freigther -> freighter
{'freighter'}
caringly -> daringly
{'carpingly', 'carkingly', 'daringly', 'scaringly'}
notedious -> notorious
{'tedious', 'notorious', 'untedious'}
varity -> variety
{'verity', 'vanity', 'parity', 'varify', 'rarity', 'carity', 'varsity', 'variety'}
potentual -> potential
{'potential'}
tweeks -> weeks
{'tweeds', 'weeks', 'tweek', 'tweens', 'tweets', 'tweaks'}
awear -> wear
{'anear', 'aweary', 'ayear', 'abear', 'swear', 'awea', 'arear', 'wear', 'afear'}
isuse -> issue
{'misuse', 'issue', 'disuse', 'isus'}
prity -> pity
{'parity', 'privy', 'pridy', 'prety', 'pricy', 'purity', 'primy', 'pity'}
obease -> obese
{'obese', 'olease'}
absolutly -> absolutely
{'absolutely'}
possitive -> positive
{'positive'}
truely -> truly
{'trully', 'truly', 'ruely'}
framerate -> frame rate
{'frame rate'}
rekt -> rest
{'rect', 'rket', 'reit', 'rept', 

hotrodw -> hotrod
{'hotrod', 'hotrods'}
withough -> without
{'withouts', 'without', 'though', 'although', 'withouta'}
recomend -> recommend
{'recommend'}
havnt -> haunt
{'hadnt', 'haunt', 'hant', "havn't", 'havant', 'hasnt', 'havent'}
alot -> lot
{'blot', 'alost', 'slot', 'clot', 'alit', 'abot', 'alout', 'alo', 'lot', 'aot', 'alon', 'salot', 'alto', 'alt', 'aloe', 'alop', 'alod', 'aloft', 'aloo', 'flot', 'alcot', 'allot', 'alow', 'plot', 'ilot'}
alot -> lot
{'blot', 'alost', 'slot', 'clot', 'alit', 'abot', 'alout', 'alo', 'lot', 'aot', 'alon', 'salot', 'alto', 'alt', 'aloe', 'alop', 'alod', 'aloft', 'aloo', 'flot', 'alcot', 'allot', 'alow', 'plot', 'ilot'}
amobile -> mobile
{'amabile', 'mobile'}
veriety -> variety
{'verity', 'variety'}
expeirince -> experience
{'experience'}
basicly -> basically
{'basify', 'basic', 'bailly', 'basial', 'basils', 'basally', 'basile', 'easily', 'basics', 'bassly', 'busily', 'basely', 'baily', 'basicity', 'basically', 'basil', 'vasily'}
dollers -> dollars


...

painitng -> painting
{'paining', 'painting'}
goty -> got
{'gota', 'geoty', 'gott', 'goy', 'gouty', 'gotz', 'coty', 'gote', 'gory', 'got', 'goety', 'doty', 'gony', 'goto', 'goby', 'roty', 'toty', 'goth', 'goaty', 'gots'}
molyneux -> None
None
syndrom -> syndrome
{'syndrome'}
ishiny -> shiny
{'shiny'}
syndrom -> syndrome
{'syndrome'}
lowrey -> lowery
{'towrey', 'lowney', 'lowry', 'lowery'}
lowrey -> lowery
{'towrey', 'lowney', 'lowry', 'lowery'}
draggin -> dragging
{'dragging'}
ontop -> onto
{'untop', 'onto', 'onto-'}
painitng -> painting
{'paining', 'painting'}
disapointment -> disappointment
{'disappointment'}
basicaly -> basically
{'basically'}
foward -> forward
{'coward', 'toward', 'howard', 'forward', 'froward'}
underperformed -> None
None
critisized -> criticized
{'criticized'}
ontop -> onto
{'untop', 'onto', 'onto-'}
tabed -> taped
{'taberd', 'taked', 'tawed', 'tabued', 'tabet', 'taped', 'tabel', 'tased', 'abed', 'sabed', 'taled', 'taber', 'tared', 'tabes', 'tabid', 'tubed', 'tame

scrren -> screen
{'sorren', 'screen'}
definetly -> definetely
{'definetely', 'definedly'}
neato -> neat
{'neat', 'nesto', 'neath', 'neats', 'neto', 'nato', 'seato', 'nemato'}
repetive -> repletive
{'repletive'}
moreso -> mores
{'moresk', 'mores', 'moreno', 'moresco'}
recieve -> receive
{'receive', 'relieve'}
gametime -> sometime
{'gamelike', 'gamete', 'facetime', 'waketime', 'sometime', 'gamesome', 'gametic'}
repetetive -> repetitive
{'repetitive'}
starforge -> starford
{'starford'}
nbsp -> bsp
{'tbsp', 'bsp', 'nasp', 'nbp', 'nsp', 'nbs'}
whereever -> wherever
{'wherever', 'whereover'}
liklihood -> likelihood
{'likelihood', 'livlihood'}
smh -> suh
{'sah', 'smt', 'mh', 'samh', 'smb', 'msh', 'smi', 'smo', 'sth', 'sdh', 'tmh', 'sh', 'smoh', 'sml', 'sm', 'smd', 'shm', 'cmh', 'smm', 'shh', 'mmh', 'sch', 'soh', 'smg', 'smr', 'sms', 'sma', 'sme', 'smp', 'smc', 'suh'}
realtime -> mealtime
{'mealtime', 'real-time'}
minate -> minute
{'ominate', 'miniate', 'minae', 'micate', 'aminate', 'misate', 

ouside -> outside
{'outside', 'obside', 'onside', 'oside'}
sentinal -> sentinel
{'sentinel', 'dentinal'}
respawned -> spawned
{'spawned', 'respaded', 'respaced', 'resawed', 'unspawned'}
nevr -> never
{'never', 'neve', 'nebr', 'near', 'ner', 'nevo', 'nev', 'nev.', 'neva', 'neer', 'nevi'}
overhyped -> overhype
{'overhype', 'overhoped', 'overtyped'}
recieved -> received
{'received', 'relieved'}
mutiplayer -> multiplayer
{'multiplayer'}
excourage -> encourage
{'encourage'}
overhyped -> overhype
{'overhype', 'overhoped', 'overtyped'}
overhyped -> overhype
{'overhype', 'overhoped', 'overtyped'}
overhyped -> overhype
{'overhype', 'overhoped', 'overtyped'}
config -> confit
{'confix', 'confit'}
antialiasing -> None
None
screenshots -> screenshot
{'screenshot'}
bsolutely -> absolutely
{'absolutely'}
framerate -> frame rate
{'frame rate'}
tralier -> trailer
{'trailer'}
falsly -> falsely
{'falsely', 'fally'}
aggrivating -> aggravating
{'aggravating'}
reccomend -> recommend
{'recommend', 're-commen

{'recommend', 're-commend'}
repetetive -> repetitive
{'repetitive'}
diffrent -> different
{'different'}
intereset -> interest
{'interset', 'interest'}
sandplanets -> None
None
spoileralert -> None
None
altas -> altar
{'antas', 'artas', 'alas', 'altos', 'altaf', 'almas', 'alias', 'altis', 'altus', 'altay', 'aldas', 'alfas', 'altars', 'alts', 'altai', 'alta.', 'atlas', 'albas', 'altes', 'altar', 'algas', 'alta'}
revieuws -> reviews
{'reviews'}
puthole -> pothole
{'pithole', 'pothole'}
isurvival -> survival
{'survival'}
rng -> ring
{'ing', 'ang', 'rna', 'ring', 'rn', 'ong', 'rngc', 'rtg', 'rong', 'reg', 'rnr', 'rne', 'ung', 'rnd', 'rpg', 'reng', 'rig', 'rhg', 'eng', 'rang', 'tng', 'rog', 'lng', 'rug', 'rag', 'rlg', 'rg', 'rung', 'ng'}
rng -> ring
{'ing', 'ang', 'rna', 'ring', 'rn', 'ong', 'rngc', 'rtg', 'rong', 'reg', 'rnr', 'rne', 'ung', 'rnd', 'rpg', 'reng', 'rig', 'rhg', 'eng', 'rang', 'tng', 'rog', 'lng', 'rug', 'rag', 'rlg', 'rg', 'rung', 'ng'}
repititious -> repetitious
{'repetitiou

disire -> desire
{'desire'}
crysis -> crisis
{'clysis', 'crasis', 'chrysis', 'crisis'}
implimented -> implemented
{'implemented'}
sneitre -> spectre
{'sneathe', 'spectre', 'nitre', 'seiter', 'treitre', 'snithe', 'snite', 'unmitre', 'sceptre'}
doign -> doing
{'doin', 'deign', 'soign', 'doing', 'doig', 'doigt', 'coign'}
customise -> customize
{'customised', 'customize'}
screenshots -> screenshot
{'screenshot'}
framerates -> frame rate
{'frame rate'}
unproffessional -> unprofessional
{'unprofessional'}
becausse -> because
{'because', 'becasse'}
disappointement -> disappointment
{'disappointment'}
taht -> that
{'tat', 'taft', 'baht', 'tait', 'tatt', 'tart', 'tant', 'tahu', 'aht', 'tath', 'tact', 'tahr', 'tah', 'that', 'taut', 'taha', 'takt'}
afk -> ask
{'aik', 'afm', 'af.', 'ark', 'alk', 'afb', 'afs', 'afg', 'aff', 'ak', 'aft', 'af', 'afi', 'afl', 'afr', 'atk', 'awk', 'afa', 'ask', 'afsk', 'af-', 'afp', 'auk', 'jfk', 'ack', 'ank', 'afc', 'afe', 'afd'}


In [6]:
spell_data[spell_data['wordtoken'] != spell_data['correct']]

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correct,correctmapping
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...",<function clean_content at 0x0000019F77852200>,"['this', 'be', 'my', 'review', 'prior', 'to', ..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...",<function clean_content at 0x0000019F77852200>,"['when', 'the', 'game', 'first', 'come', 'out'..."
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...",<function clean_content at 0x0000019F77852200>,"['if', 'that', 'do', 'not', 'prove', 'this', '..."
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...",<function clean_content at 0x0000019F77852200>,"['the', 'developer', 'completely', 'lie', 'abo..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...",<function clean_content at 0x0000019F77852200>,"['clearly', 'have', 'very', 'strong', 'feel', ..."
...,...,...,...,...,...,...,...,...,...,...,...
163842,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",Only about 3 or 4 hours of it was me actually ...,-1,0,22.3,only about or hour of it be me actually enjoy ...,"['only', 'about', 'or', 'hour', 'of', 'it', 'b...",<function clean_content at 0x0000019F77852200>,"['only', 'about', 'or', 'hour', 'of', 'it', 'b..."
163843,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The free roam exploration runs dry very quickl...,-1,0,41.1,the free roam exploration run dry very quickly...,"['the', 'free', 'roam', 'exploration', 'run', ...",<function clean_content at 0x0000019F77852200>,"['the', 'free', 'roam', 'exploration', 'run', ..."
163844,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...","Overall, No Man's Sky is a lazy game not even ...",-1,0,9.7,overall no man be sky be a lazy game not even ...,"['overall', 'no', 'man', 'be', 'sky', 'be', 'l...",<function clean_content at 0x0000019F77852200>,"['overall', 'no', 'man', 'be', 'sky', 'be', 'l..."
163845,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The game's cycle is essentially go to a planet...,-1,0,23.7,the game be cycle be essentially go to a plane...,"['the', 'game', 'be', 'cycle', 'be', 'essentia...",<function clean_content at 0x0000019F77852200>,"['the', 'game', 'be', 'cycle', 'be', 'essentia..."


In [7]:
spell_data['correct'] = spell_data['correct']
spell_data

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correct,correctmapping
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...",<function clean_content at 0x0000019F77852200>,"['this', 'be', 'my', 'review', 'prior', 'to', ..."
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...",<function clean_content at 0x0000019F77852200>,"['when', 'the', 'game', 'first', 'come', 'out'..."
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...",<function clean_content at 0x0000019F77852200>,"['if', 'that', 'do', 'not', 'prove', 'this', '..."
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...",<function clean_content at 0x0000019F77852200>,"['the', 'developer', 'completely', 'lie', 'abo..."
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...",<function clean_content at 0x0000019F77852200>,"['clearly', 'have', 'very', 'strong', 'feel', ..."
...,...,...,...,...,...,...,...,...,...,...,...
163842,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",Only about 3 or 4 hours of it was me actually ...,-1,0,22.3,only about or hour of it be me actually enjoy ...,"['only', 'about', 'or', 'hour', 'of', 'it', 'b...",<function clean_content at 0x0000019F77852200>,"['only', 'about', 'or', 'hour', 'of', 'it', 'b..."
163843,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The free roam exploration runs dry very quickl...,-1,0,41.1,the free roam exploration run dry very quickly...,"['the', 'free', 'roam', 'exploration', 'run', ...",<function clean_content at 0x0000019F77852200>,"['the', 'free', 'roam', 'exploration', 'run', ..."
163844,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...","Overall, No Man's Sky is a lazy game not even ...",-1,0,9.7,overall no man be sky be a lazy game not even ...,"['overall', 'no', 'man', 'be', 'sky', 'be', 'l...",<function clean_content at 0x0000019F77852200>,"['overall', 'no', 'man', 'be', 'sky', 'be', 'l..."
163845,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The game's cycle is essentially go to a planet...,-1,0,23.7,the game be cycle be essentially go to a plane...,"['the', 'game', 'be', 'cycle', 'be', 'essentia...",<function clean_content at 0x0000019F77852200>,"['the', 'game', 'be', 'cycle', 'be', 'essentia..."


In [8]:
spell_data.to_csv('./processing_datasets/preprocessed_word_correct.csv',index=False)